In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot
import time

from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Utils.DataReader import load_urm, load_icm, load_target
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

In [3]:
URM_all = load_urm()

URM_train = sps.load_npz('Dataset/Split/URM_train.npz')
URM_test =  sps.load_npz('Dataset/Split/URM_test.npz')
URM_validation = sps.load_npz('Dataset/Split/URM_validation.npz')

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1553 ( 3.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 915 ( 2.2%) Users that have less than 1 test interactions


In [4]:
recommender_class = SLIMElasticNetRecommender

In [5]:
import os

output_folder_path = "Experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 50
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"
cutoff_to_optimize = 10

In [6]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "topK": Integer(1,800),
    "l1_ratio": Real(low = 0.001, high = 0.01, prior = 'log-uniform'),
    "alpha": Real(low = 0.001, high = 0.005, prior = 'log-uniform'),
    #"positive_only": Categorical([True]),
}

In [7]:
earlystopping_keywargs = {"validation_every_n": 15,
                          "stop_on_validation": True,
                          "evaluator_object": evaluator_validation,
                          "lower_validations_allowed": 5,
                          "validation_metric": metric_to_optimize,
                          }

In [8]:
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

#create a bayesian optimizer object, we pass the recommender and the evaluator
hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                           evaluator_validation=evaluator_validation)

In [9]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

#provide data needed to create instance of model (one on URM_train, the other on URM_all)
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS ={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_all],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS ={}
)

In [10]:
hyperparameterSearch.search(recommender_input_args = recommender_input_args,
                            recommender_input_args_last_test = recommender_input_args_last_test,
                            hyperparameter_search_space = hyperparameters_range_dictionary,
                            n_cases = n_cases,
                            n_random_starts = n_random_starts,
                            save_model = "last",
                            output_folder_path = output_folder_path, # Where to save the results
                            output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                            metric_to_optimize = metric_to_optimize,
                            cutoff_to_optimize = cutoff_to_optimize,
                            )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 612, 'l1_ratio': 0.006189663115319857, 'alpha': 0.00442212948342893}


SLIMElasticNetRecommender: Processed 4862 (19.8%) in 5.00 min. Items per second: 16.20


SLIMElasticNetRecommender: Processed 10300 (42.0%) in 10.00 min. Items per second: 17.16


SLIMElasticNetRecommender: Processed 15489 (63.2%) in 15.00 min. Items per second: 17.21


SLIMElasticNetRecommender: Processed 20685 (84.4%) in 20.00 min. Items per second: 17.23


...

EvaluatorHoldout: Processed 40076 (100.0%) in 47.76 sec. Users per second: 839


SearchBayesianSkopt: Config 23 is suboptimal. Config: {'topK': 572, 'l1_ratio': 0.004130427591029129, 'alpha': 0.001970858754371783} - results: PRECISION: 0.0423969, PRECISION_RECALL_MIN_DEN: 0.0799217, RECALL: 0.0766212, MAP: 0.0195999, MAP_MIN_DEN: 0.0370338, MRR: 0.1418891, NDCG: 0.0727588, F1: 0.0545884, HIT_RATE: 0.3030492, ARHR_ALL_HITS: 0.1659474, NOVELTY: 0.0043792, AVERAGE_POPULARITY: 0.1742649, DIVERSITY_MEAN_INTER_LIST: 0.9581220, DIVERSITY_HERFINDAHL: 0.9958098, COVERAGE_ITEM: 0.1690129, COVERAGE_ITEM_HIT: 0.0543926, ITEMS_IN_GT: 0.9868201, COVERAGE_USER: 0.9626943, COVERAGE_USER_HIT: 0.2917437, USERS_IN_GT: 0.9626943, DIVERSITY_GINI: 0.0235058, SHANNON_ENTROPY: 9.2723019, RATIO_DIVERSITY_HERFINDAHL: 0.9961390, RATIO_DIVERSITY_GINI: 0.0464653, RATIO_SHANNON_ENTROPY: 0.6873325, RATIO_AVERAGE_POPULARITY: 3.4278306, RATIO_NOVELTY: 0.2948458, 



Iteration No: 24 ended. Search finished for the next optimal point.
Time taken: 1666.4386
Function value obtained: -0.0196
Current minimum: -0.0198
Iteration No: 25 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 102, 'l1_ratio': 0.00249647785042239, 'alpha': 0.0014453141415612066}


SLIMElasticNetRecommender: Processed 3842 (15.7%) in 5.00 min. Items per second: 12.80


SLIMElasticNetRecommender: Processed 8206 (33.5%) in 10.00 min. Items per second: 13.67


In [ ]:
from Recommenders.DataIO import DataIO

#explore the results of the search
data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

search_metadata.keys()

In [ ]:
hyperparameters_df = search_metadata["hyperparameters_df"]
hyperparameters_df

In [ ]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df

In [ ]:
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters